In [ ]:
#debug pandas not importing
import sys
print(sys.executable)
#taking the information from the line above -- directly import pandas module in here -- this is some wizardry
! /Users/Amy/anaconda3/python.exe -m pip install pandas

Step 1 - import data into juypter notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Step 2 - Use matplotlib to make a barplot of the data and check for outliers

In [ ]:
#import the data
df = pd.read_csv("DESEQ2NormalizedData.csv", index_col=0)
df.head()

df.shape #returns dimensions of the data

In [ ]:
#swaps the x and y values
data = df.transpose()
data.shape

In [ ]:
#print(plt.style.available) list of available displays

plt.style.use('fast')
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])

fig.set_size_inches(50, 10.5)

plt.boxplot(data)
plt.xticks(rotation = 90)

plt.xlabel("Samples", size = 32)
plt.ylabel("Expression", size = 32)
plt.title("Normalization Barplot", size = 38)

x_ticks = list(df.index)
ax.set_xticklabels(x_ticks)

plt.show()

Step 3 - remove outliers; none exist
Step 4 - find library to convert gene ids into gene symbols - perhaps an R package or online resource?

Possible resource - https://www.researchgate.net/post/How-to-map-probe-ID-with-gene-symbol-in-GEO-dataset
Dataset - https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE125583
Official gene symbols (I think) - https://www.genenames.org/tools/search/

Steps for 4: 

Utilize API provided by NCBI to scrape gene information from data

Source: https://www.ncbi.nlm.nih.gov/home/develop/api/

Entrez appears to be the most promising since it offers databases for genes which is the one we're targeting.

Sample of what we want to achieve: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gene&id=100093631
^From that, if I get the <Name> section of the gene and combine them together, then it's all set

Always returns xml file. 

Template: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gene&id=[insert custom id parsed from data here]

In [ ]:
# "test" function to grab gene information

#need to write a script that automatically runs through the urls for me
import requests as r

# genes = "100093631"
#response = r.get(f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gene&id={genes}") #created a template literal with variable that functions as the value
#print(response)

query = {'db': 'gene', 'id': '100093631,1,2,3,4,5,6,7,8,9'}
response = r.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?', params=query)

#parse the xml file
import xml.etree.ElementTree as ET
root = ET.fromstring(response.content)
print(root[0][1][0].text) # <- this gives me the first one... woot!
print(root[0][2][0].text) # <- this is the next one!
print(root[0][3][0].text) # <- this gives the third one!
print(root[0][4][0].text) # <- this gives the fourth one!
print(root[0][5][0].text) # <- this gives the fifth one!
print(root[0][6][0].text) # <- this gives the sixth one!
print(root[0][7][0].text) # <- this gives the seventh one!
print(root[0][8][0].text) # <- this gives the eighth one!
print(root[0][9][0].text) # <- this gives the ninth (nineth) one!
print(root[0][10][0].text) # <- this gives the tenth one!

References: 
- Downloading Document Summaries: https://www.ncbi.nlm.nih.gov/books/NBK25500/#chapter1.Downloading_Document_Summaries
- API Requests tutorial: https://www.nylas.com/blog/use-python-requests-module-rest-apis/
- Using ElementTree: https://www.edureka.co/blog/python-xml-parser-tutorial/#findingelements

In [ ]:
# grab gene ids from csv file and pasted it into ids.txt
# Goal is to eventually create a dictionary that links the gene id with the gene symbol

import time  # will be useful when trying to puase the program for a while
import csv
file = open('DESeq2NormalizedData.csv')
csvreader = csv.reader(file)
header = []
header = next(csvreader) 
# header contains all of the items in the first row
# remove the first one since that's just a ''
header = header[1:]

# write into the file and then drag it onto the other row later on
with open('key.csv', 'w', newline="") as file:
    csvwriter = csv.writer(file)  # create csvwriter object
    csvwriter.writerow(header)  # write the header (aka the gene ids)

    # have to get the data before I write it
    data = []
    tempIDHolder = []
    # create a dictionary that links the gene id with the gene symbol
    for id in header:
        tempIDHolder.append(id)
        if (len(tempIDHolder) >= 10):
        # request a downloadable summary, etc.
            # first convert the tempIDHolder into actual id value
            id = tempIDHolder[0]
            counter = 0
            for item in tempIDHolder:
                if counter == 0:
                    counter += 1
                else :
                    id = id + ',' + item # hopefully this gets everything?
            # print(id)
            query = {'db': 'gene', 'id': id}
            response = r.get(
                'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?', params=query)
            root = ET.fromstring(response.content)
            data.append(root[0][1][0].text)
            data.append(root[0][2][0].text)
            data.append(root[0][3][0].text)
            data.append(root[0][4][0].text)
            data.append(root[0][5][0].text)
            data.append(root[0][6][0].text)
            data.append(root[0][7][0].text)
            data.append(root[0][8][0].text)
            data.append(root[0][9][0].text)
            data.append(root[0][10][0].text)
            # make it pause or else the API will yell at me (it was like 3 requests per second if I remember correctly)
            # make it sleep for half a second... therefore it should be like 2 requests per second
            # print(root[0][1][0].text)
            time.sleep(.5)
            # reset tempIDHolder
            tempIDHolder.clear()

    csvwriter.writerow(data)

In [ ]:
#find the gene ids that I am still missing (since I did it by factors of 10) and manually modify the key.csv
print(len(header))
# result: 16232, so that means I only have to get two, phew!

In [ ]:
# double checking my computer's capability to handle large array sizes - source: https://stackoverflow.com/questions/855191/how-big-can-a-python-list-get#:~:text=According%20to%20the%20source%20code,PY_SSIZE_T_MAX%2Fsizeof(PyObject*)%20.&text=On%20a%20regular%2032bit%20system,bit%20system%20is%20536%2C870%2C912%20elements.
import six
print(six.MAXSIZE)
# result: 9223372036854775807, so yeah it should be fine

Resources:
- Manipulating csv file to get just the header: https://www.analyticsvidhya.com/blog/2021/08/python-tutorial-working-with-csv-file-for-data-science/
- How to manipulate python dictionaries: https://www.geeksforgeeks.org/python-dictionary/

Replace DF ids with Gene Symbols

In [ ]:
# import conversion library
translate = pd.read_csv("key.csv")
translate.head()

In [ ]:
colNames = list(translate.iloc[0, :])

DataTot = df.set_axis(colNames, axis=1, inplace=False)
DataTot.head()

Select genes to focus on

Genes extracted from DE here: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE132651

In [ ]:
# I used a Google script and ripped off an array and just copy pasted it here. Is it clean? Haha no
gene_list = ["MMP1", "TFPI2", "ANGPT2", "SULF1", "SULF1", "FABP4", "ANGPT2", "IL1RL1", "TFPI2", "ADGRF5", "FST", "VAMP8", "SULF1", "MRC1", "TOX", "BASP1", "EMCN", "LYVE1", "COL4A5", "MMP10", "ANKRD1", "TACSTD2", "GULP1", "RASGRP3", "LEPROT///LEPR", "GULP1", "NUPR1", "PBK", "SULT1E1", "ADGRF5", "PROS1", "EPB41L3", "ALDH1A1", "PFN2", "FABP5", "TSPAN13", "FGF13", "LIMCH1", "GMFG", "IL1R1", "PDE2A", "SRSF6", "TGFB2", "LYN", "CLU", "LBH", "MIR6787///SLC16A3", "EPB41L3", "CHST15", "ANOS1", "TIMP3", "RRM2", "RGCC", "MIR6787///SLC16A3", "EPB41L3", "FST", "HSD17B2", "GULP1", "THSD7A", "SLC6A15", "LTBP1", "COL13A1", "LIMCH1", "KRT7", "ANXA3", "MAD2L1", "LAMP3", "DYSF", "SLC35F6///CENPA", "CLEC2B", "LDB2", "LTBP1", "LYVE1", "DKK1", "RPS4Y1", "SLIT2", "DPYSL3", "FBLN5", "NRP2", "CXCR4", "C10orf10", "CLU", "NRG1", "CSGALNACT1", "HJURP", "ASPM", "PLA2G4C", "ADIRF", "TOP2A", "BST2", "CD44", "HLA-B", "LIMCH1", "TGFB2", "DLGAP5", "SULT1B1", "SACS", "COL3A1", "HMMR", "LOC101930400///AKR1C2", "TIMP3", "IFI27", "BMP4", "NID1", "UCHL1", "NDC1", "SHCBP1", "LYN", "EMP3", "CCND2", "CCL15-CCL14///CCL14", "CDC20", "MAFB", "GPX3", "AURKA", "PTTG1", "RFC3", "BCAT1", "RBP1", "MEIS2", "CCNB1", "SH3BP5", "VCAM1", "CKS1B", "PRC1", "JUP", "GJA4", "NCAPG", "LYPD1", "BCHE", "TMEM140", "IGFBP2", "CCND2", "HOXA10-HOXA9///MIR196B///HOXA9", "AKR1C1", "TNFSF10", "PODXL", "IL13RA2", "SPC25", "UBE2S", "GYPC", "NDC80", "ABCG1", "CEP55", "EIF1AY", "GATA6", "GPRC5A", "PDPN", "TNFSF10", "CD36", "RELN", "CCNA2", "CSF2RB", "ANPEP", "CDK1", "GJA4", "MT1M", "TRPV2", "RRM2", "CLU", "GPX3", "TGFBR2", "CCNA1", "SMC2", "UBD///GABBR1", "TGFB2", "ADD3", "FHL1", "KIAA0101", "MELK", "SQRDL", "PIEZO2", "RRP15", "DEPDC1", "XIST", "TPD52", "SOX18", "OIP5", "MFAP2", "MPZL2", "TMPO", "CCNA2", "TPX2", "TRIP13", "PCDH7", "PLPP3", "PCNA", "PTPRN2", "POSTN", "NRP2", "BACE2", "NPTX2", "DKK3", "PLOD2", "TGFBR2", "SPRY1", "HSPD1", "MT1X", "KIF20A", "CENPN" ]

len(gene_list)

In [ ]:
# see which ones match
colNames = list(DataTot.columns)

geneOverlap = list(set(gene_list).intersection(colNames))
len(geneOverlap)

In [ ]:
selectDf = DataTot.loc[:, geneOverlap[0:100]]
# create an excel file of the dataframe
# selectDf.to_excel("completeDataset.xlsx")
selectDf.head()

Make dataset

In [ ]:
# create 70/30 split
shuffledData = selectDf.sample(frac=1, random_state=42)
shuffledData.head()

In [ ]:
x, y = shuffledData.shape
splitThreshold = round(x*.7)

TrainData = shuffledData.iloc[0:splitThreshold, :]
TestData = shuffledData.iloc[splitThreshold:, :]

In [ ]:
# create input and output set (50/50) for now, but may change later
# randomize columns
tempShuffleDf = TrainData.T
tempShuffleDf = tempShuffleDf.sample(frac=1, random_state=500)
TrainData = tempShuffleDf.T
tempShuffleDf = TestData.T
tempShuffleDf = tempShuffleDf.sample(frac=1, random_state=500)
TestData = tempShuffleDf.T

# split the data
x_train = TrainData.iloc[:, 0:50]
y_train = TrainData.iloc[:, 50:]

x_test = TestData.iloc[:, 0:50]
y_test = TestData.iloc[:, 50:]

# print(tempShuffleDf.T) #note .T transposes/reverses the data
y_test.head()

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

Train Random Forest

I used classification first simply because it was the first one lol... future Amy here. Turns out I should be using regression instead or else I run into a 'continuous-multioutput' error... 

In [ ]:
# importing sklearn (and pandas again) because my computer doesn't recognize it
! /Users/Amy/anaconda3/python.exe -m pip install sklearn

In [ ]:
# ml import
from sklearn.ensemble import RandomForestRegressor

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(df.iloc[0, 0])
print('---------')
print(selectDf.head(0).columns)

In [ ]:
# instantiate model and train
reg = RandomForestRegressor(n_estimators=400, random_state=42)
reg.fit(x_train, y_train) #nah this should work... but it just doesn't quite match the actual thing nicely

Test Random Forest

In [ ]:
y_pred = reg.predict(x_test)

In [ ]:
print(y_test.shape, y_pred.shape) #wtf y_pred is already a numpyarray whereas y_test is the one that isn't
ny_test = y_test.to_numpy()
print(y_pred[:, 0]) #only the predicted values for the first gene
print(ny_test[:, 0]) #only the actual values for the first gene

In [ ]:
df2 = pd.DataFrame()
df2['Predicted'] = y_pred[:, 0]
df2['Actual'] = ny_test[:, 0]

df2

In [ ]:
print(y_pred[:, 0].tolist())
print(ny_test[:, 0].tolist())

In [ ]:
#experimental function to find the percent difference error
predicted = [7.090191573100002, 7.458117004670004, 8.600444416470005, 7.788848798920004, 8.40825437213, 7.020327500409999, 8.818133271350002, 7.8266646558799975, 6.9378079908900006, 6.872748400490001, 7.506390345189996, 8.676429359800004, 8.554343255570004, 7.166067258440004, 8.747529485299998, 7.968960927520005, 7.452849082630001, 7.761508404270005, 8.320082494520001, 7.971978045290004, 6.697892382179999, 8.949659341020002, 8.5578288482, 8.959041092180001, 8.720058047870003, 9.099238304330001, 6.597845043869999, 8.771403687159996, 8.97738436032, 9.337057343459998, 8.615097380400004, 8.379626407890001, 6.837338580420003, 6.728788978850002, 6.72586007986, 8.569162017440004, 8.760991396020003, 6.398245022990002, 8.775553368840006, 8.599701066330004, 6.706699092590001, 7.119638600939998, 7.276036668970002, 8.70239843987, 7.404154973279999, 8.162479427099996, 6.613187798319999, 8.779790491420004, 8.06791024708, 7.6901312955200005, 6.943329752419999, 7.805397315200001, 7.597572052499999, 7.180362284900005, 8.814388118280004, 8.770979079269997, 8.452227193040002, 8.525045438540001, 7.466116578500001, 6.994484173059999, 7.585299090140003, 8.753943669680002, 8.870920817819997, 7.499310703240001, 9.286605262920002, 7.215856328890001, 9.01209597088, 8.669163431630004, 7.186433201240003, 7.302876306980005, 8.488108625080002, 7.504864159039996, 7.927865327730001, 6.44979854136, 7.310411780489999, 8.626030034680007, 8.696088902549997, 7.079432411410002, 8.071684938880002, 8.670569142679998, 7.816587408060005, 8.141853975380002, 8.353289437620003, 8.788246874929996, 8.285601362710002, 8.440699586870002, 8.812269140540002]
actual = [6.897812565, 7.338213509, 8.702359981, 7.948689744, 8.542811765, 6.837543279, 9.269233796, 8.330263567, 6.537387668, 7.477668772, 7.705141327, 8.388086398, 9.047330902, 7.177687891, 10.07887491, 9.96112512, 8.251808157, 6.519026254, 8.593129162, 8.215544999, 7.127999315, 10.67614923, 8.178851803, 9.602720917, 8.661232602, 9.332002144, 5.497343234, 8.824692683, 9.209058607, 10.23649005, 9.169253621, 8.778581143, 7.461903599, 5.902414739, 6.728518061, 9.278008069, 8.990431014, 6.179569244, 9.553214953, 9.109478527, 5.240724147, 6.385636127, 7.785670875, 7.881095765, 7.66056203, 7.875984985, 6.533125215, 8.506735274, 8.547510836, 7.644934135, 7.248378845, 8.506008575, 7.605692603, 7.694407888, 9.458641893, 8.627903977, 9.76776567, 8.207819731, 7.342463366, 7.43472634, 8.535281792, 8.828590939, 9.350347751, 7.530876833, 9.484744463, 7.558263108, 9.522007793, 9.427759537, 8.263312582, 7.284029873, 8.352555076, 6.142537446, 8.535022869, 5.554148673, 7.672479957, 8.185338403, 8.421832893, 7.733339952, 9.832547685, 8.320524407, 8.941760952, 9.285487465, 9.122246483, 8.718260929, 8.794853801, 8.622207487, 10.1410804]

# find the percent difference of each value, add them up and divide by 87 to find average percent difference
averagePercentDifference = 0
for index in range(87):
    # percent difference equation: |pred-actual|/actual
    averagePercentDifference += abs(predicted[index]-actual[index])/actual[index] * 100
# divided by 87
averagePercentDifference /= 87

averagePercentDifference

# Combining the previous steps to check reliability and run tests repeatedly

In [ ]:
# run this test multiple times... because I am not convinced of the results
import pandas as pd
import random
# ml import
from sklearn.ensemble import RandomForestRegressor

averagePercentDifference = 0
numTrials = 50
for i in range(numTrials):
    # convert the completeDataset.xlsx into a pandas dataframe because I don't want to redo the previous steps over and over again every time I reopen the notebook
    selectDf = pd.read_excel("completeDataset.xlsx", index_col=0)

    randNum = random.randrange(500)
    # create 70/30 split of training and testing datasets
    shuffledData = selectDf.sample(frac=1, random_state=42)

    x, y = shuffledData.shape
    splitThreshold = round(x*.7)

    TrainData = shuffledData.iloc[0:splitThreshold, :]
    TestData = shuffledData.iloc[splitThreshold:, :]

    # create input and output set (50/50) for now, but may change later
    # randomize columns
    tempShuffleDf = TrainData.T
    tempShuffleDf = tempShuffleDf.sample(frac=1, random_state=randNum)
    TrainData = tempShuffleDf.T
    tempShuffleDf = TestData.T
    tempShuffleDf = tempShuffleDf.sample(frac=1, random_state=randNum)
    TestData = tempShuffleDf.T

    splitIndex = 80 #first {splitIndex} will be used as input variables

    # split the data
    x_train = TrainData.iloc[:, 0:splitIndex]
    y_train = TrainData.iloc[:, splitIndex:]

    x_test = TestData.iloc[:, 0:splitIndex]
    y_test = TestData.iloc[:, splitIndex:]

    # instantiate model and train
    reg = RandomForestRegressor(n_estimators=100, random_state=42)
    reg.fit(x_train, y_train)

    # actually train the model
    y_pred = reg.predict(x_test)
    ny_test = y_test.to_numpy()

    x, y = y_pred.shape
    for row in range(x):
        for col in range(y):
            # calculate the percent difference for the value
            averagePercentDifference += abs(y_pred[row][col] - ny_test[row][col])/ny_test[row][col]
    averagePercentDifference /= (x*y)

averagePercentDifference*100

Tuning the hyperparameters - source: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
from pprint import pprint

print('Parameters currently in use: \n')
pprint(reg.get_params())

# Cross Validation Testing

In [ ]:
# from the article it states that the most important hyperparameters is the n_estimators (# of trees), number of features (max_features)
# possible adjustments to be made and explanation
'''
n_estimators = number of trees in the foreset
max_features = max number of features considered for splitting a node
max_depth = max number of levels in each decision tree
min_samples_split = min number of data points placed in a node before the node is split
min_samples_leaf = min number of data points allowed in a leaf node
bootstrap = method for sampling data points (with or without replacement)
'''
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)
# total of 4320 possible settings

Random Search Training

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)
# rf_pred = rf_random.predict(x_test) # something new I added... maybe check it out later

rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    errors = errors.to_numpy()
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors))) #idk if this works as intended ngl
    print('Accuracy = {:0.2f}%.'.format(np.mean(accuracy.to_list())))
    
    return accuracy
# base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
# base_model.fit(x_train, y_train)
# base_accuracy = evaluate(base_model, x_test, y_test)

base_accuracy = evaluate(reg, x_test, y_test)

print()

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test, y_test)

# Grid Search with Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [50, 100, 150], #why is it these values... idk myself ngl
    'max_features': [13, 15, 17, 19, 21], #idk for these values either tbh... maybe research later
    'min_samples_leaf': [1],
    'min_samples_split': [2],
    'n_estimators': [100, 200, 300, 400, 500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# fit the grid search to the data
grid_search.fit(x_train, y_train)
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, x_test, y_test)

# Analyze the Results

In [ ]:
from sklearn.metrics import mean_squared_error

# multidimensional RMSE calculation
def MDRMSE(predy, testy): # multi-dimensional RMSE
    # rows are samples
    # columns are genes
    # iterate through each sample and take gene distribution calculate RMSE and add to list

    # should be iterating through every predicted y value instead, fixed it
    x, y = predy.shape
    allRMSE = []
    
    for col in range(y):
        pred = predy.iloc[:, col]
        test = testy.iloc[:, col]
        rmse = mean_squared_error(test, pred)
        allRMSE.append(rmse)
    
    # Calculate sum of list and divide by number of items
    averageRMSE = sum(allRMSE)/len(allRMSE)
    
    # return the aggregated RMSE
    return averageRMSE

In [ ]:
#Percent Difference Error

def PDE(y_pred, y_test):
    averagePercentDifference = 0
    x, y = y_pred.shape
    for row in range(x):
        for col in range(y):
            # calculate the percent difference for the value
            averagePercentDifference += abs(y_pred[row][col] - y_test[row][col])/y_test[row][col]
    averagePercentDifference /= (x*y)
    return averagePercentDifference*100

In [ ]:
from sklearn.metrics import r2_score

# more info about multioutput parameter: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

# R^2 function
def R2_SCORE(y_pred, y_test, modelType):
    if (modelType == 'FOREST'): # source: https://stats.stackexchange.com/questions/7357/manually-calculated-r2-doesnt-match-up-with-randomforest-r2-for-testing?noredirect=1&lq=1
        return r2_score(y_test, y_pred, multioutput='variance_weighted')
    elif (modelType == 'PLS'): # source: https://www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/statistics/regression-and-correlation/coefficient-of-determination-r-squared.html#:~:text=Solution,into%20the%20regression%20line%20equation.
        return r2_score(y_test, y_pred)
    else:
        print("The only values accepted are \'FOREST\' and \'PLS\'.")
        return #can I not add a return here?


In [ ]:
RMSerror = MDRMSE(pd.DataFrame(y_pred), y_test)
percentError = PDE(y_pred, y_test.to_numpy())
R2_1 = R2_SCORE(y_pred, y_test, 'FOREST')
R2_2 = R2_SCORE(y_pred, y_test, 'PLS')

print("Predicted values are all from initial \"Forest\" model")
print("Root Means Squared Error: ", end="")
print(RMSerror)
print("Percent Difference Error: ", end="")
print(percentError)
print("R2 SCORE - as a forest: ", end="")
print(R2_1)
print("R2 SCORE - as a PLS: ", end="")
print(R2_2)

# Create Residual Plots to show Performance of Forest is Working

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def StackedResidualPlot(y_pred, y_test):
    # conert to np array with try/except
    ny_test = y_test.to_numpy() #what's with the try/except catch method used for?

    # calculate the residuals by column (variable)
    residual = y_pred - ny_test
    # predicted y on x axis and residual on y axis
    # go iteratively over row by row -- AKA sample by sample

    numRows = 5
    numCols = 4

    f = plt.figure()
    f, axes = plt.subplots(nrows = numRows, ncols = numCols, sharex = True, sharey = True, figsize=(25, 25))

    # axis and title
    f.suptitle('Residual Plots', fontsize=42)
    f.supxlabel('Predicted Values', fontsize=38) # reference: https://stackoverflow.com/a/65135433/15073477
    f.supylabel('Residual Values', fontsize=38)

    for row in range(numRows):
        for col in range(numCols):
            axes[row][col].scatter(y_pred[row+col], residual[row+col], marker = "o") #c = getRand(100)
            # create a horizontal line!!! - source: https://pythonguides.com/matplotlib-best-fit-line/
            theta = np.polyfit(y_pred[row+col], residual[row+col], 1) # 1 represents the degree
            y_line = theta[1] + theta[0] * y_pred[row+col] # it said theta[0] * X... not sure what X represents in my code
            axes[row][col].plot(y_pred[row+col], y_line, 'r')

    # add separate colorbar axes
    # cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
    # f.colorbar(axes[0][0], cax=cbar_ax)

    plt.show()

StackedResidualPlot(y_pred, y_test)

# Create Partial Least Squares Regression

In [ ]:
# Another tutorial with cross validation: https://www.statology.org/partial-least-squares-in-python/

# ml imports
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Source/documentation: https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html

from sklearn.cross_decomposition import PLSRegression

pls2 = PLSRegression(n_components=9)
pls2.fit(x_train, y_train)

pls2_y_pred = pls2.predict(x_test)

pls2.get_params()

# Optimize Partial Least Squares - Source: https://nirpyresearch.com/partial-least-squares-regression-python/

In [ ]:
def optimise_pls_cv(X, y, n_comp):
    # Define PLS object
    pls = PLSRegression(n_components=n_comp) #should use nipals algorithm

    # Cross-validation
    y_cv = cross_val_predict(pls, X, y, cv=10)

    # Calculate scores
    r2 = r2_score(y, y_cv)
    mse = mean_squared_error(y, y_cv)
    
    return (y_cv, r2, mse, rpd)


In [ ]:
# test with 30 components
r2s = []
mses = []
x, y = y_train.shape
xticks = np.arange(1, 21)
for n_comp in xticks:
    y_cv, r2, mse, rpd = optimise_pls_cv(x_train, y_train, n_comp)
    r2s.append(r2)
    mses.append(mse)

In [ ]:
# Plot the mses
def plot_metrics(vals, ylabel, objective):
    with plt.style.context('ggplot'):
        plt.plot(xticks, np.array(vals), '-v', color='blue', mfc='blue')
        if objective=='min':
            idx = np.argmin(vals)
        else:
            idx = np.argmax(vals)
        plt.plot(xticks[idx], np.array(vals)[idx], 'P', ms=10, mfc='red')

        plt.xlabel('Number of PLS components')
        plt.xticks = xticks
        plt.ylabel(ylabel)
        plt.title('PLS')

    plt.show()

In [ ]:
plot_metrics(mses, 'MSE', 'min')
plot_metrics(r2s, 'R2', 'max')

In [ ]:
y_cv, r2, mse, rpd = optimise_pls_cv(x_test, y_test, 18) #18 depends on what the points of the previous code block show

# Evaluate Partial Least Squares

In [ ]:
RMSerror = MDRMSE(pd.DataFrame(pls2_y_pred), y_test)
percentError = PDE(pls2_y_pred, y_test.to_numpy())
R2_1 = R2_SCORE(pls2_y_pred, y_test, 'FOREST')
R2_2 = R2_SCORE(pls2_y_pred, y_test, 'PLS')

print("Predicted values are all from initial \"PLS\" Regression")
print("Root Means Squared Error: ", end="")
print(RMSerror)
print("Percent Difference Error: ", end="")
print(percentError)
print("R2 SCORE - as a forest: ", end="")
print(R2_1)
print("R2 SCORE - as a PLS: ", end="")
print(R2_2)

# Note: should also consider using RASE for error as a comparison, etc.

In [ ]:
# the results of the forest model are here:

RMSerror = MDRMSE(pd.DataFrame(y_pred), y_test)
percentError = PDE(y_pred, y_test.to_numpy())
R2_1 = R2_SCORE(y_pred, y_test, 'FOREST')
R2_2 = R2_SCORE(y_pred, y_test, 'PLS')

print("Predicted values are all from initial \"Forest\" model")
print("Root Means Squared Error: ", end="")
print(RMSerror)
print("Percent Difference Error: ", end="")
print(percentError)
print("R2 SCORE - as a forest: ", end="")
print(R2_1)
print("R2 SCORE - as a PLS: ", end="")
print(R2_2)